In [57]:
import h5py, os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from Bio import SeqIO
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [58]:
from training.naive_bayes.naive_bayes import tokenize_genes_substring

In [59]:
tokenized_sequences_test = tokenize_genes_substring("../../paperData/pM10Kb_1KTest/human_promoter_sequences_test.fasta", 3500, 13501)
tokenized_sequences_val = tokenize_genes_substring("../../paperData/pM10Kb_1KTest/human_promoter_sequences_valid.fasta", 3500, 13501)
tokenized_sequences_training = tokenize_genes_substring("../../paperData/pM10Kb_1KTest/human_promoter_sequences_train.fasta", 3500, 13501)
tokenized_sequences = np.concatenate((tokenized_sequences_test, tokenized_sequences_val, tokenized_sequences_training), axis=None)
print(len(tokenized_sequences))

tokenize_genes_substring CALLED
tokenize_genes_substring CALLED
tokenize_genes_substring CALLED
18377


In [60]:
datadir = "..\\..\\paperData\\pM10Kb_1KTest"

# Get truth data
testfile = h5py.File(os.path.join(datadir, 'test.h5'), 'r')
valfile = h5py.File(os.path.join(datadir, 'valid.h5'), 'r')
trainfile = h5py.File(os.path.join(datadir, 'train.h5'), 'r')
y = np.concatenate((testfile['detectionFlagInt'][:], valfile['detectionFlagInt'][:], trainfile['detectionFlagInt'][:]), axis=None)
print("y.shape: ", y.shape)

# exclude genes if expression is unknown i.e., 2
excluded_indices = [i for i in range(len(y)) if y[i] == 2]
y = [label for (idx, label) in enumerate(y) if idx not in excluded_indices]
tokenized_sequences = [seq for (idx, seq) in enumerate(tokenized_sequences) if idx not in excluded_indices]

# Use CountVectorizer to count k-mers
vectorizer = CountVectorizer()
#vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokenized_sequences)

# You now have a sparse matrix of k-mer counts
print("X.shape", X.shape)  # (num_sequences, num_unique_kmers)print("y.shape", y.shape)
print("len(y)", len(y))


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

y.shape:  (18377,)
X.shape (18344, 4150)
len(y) 18344


In [61]:
from scipy.sparse import vstack
absent_idx = [idx for (idx, label) in enumerate(y_train) if label == 0]
present_idx = [idx for (idx, label) in enumerate(y_train) if label == 1][:len(absent_idx)]
absent_X = X_train[absent_idx]
present_X = X_train[present_idx]
absent_y = [l for l in y_train if l == 0]
present_y = [l for l in y_train if l == 1][:len(absent_y)]
X_train_filter = vstack([absent_X, present_X])
y_train_filter = np.concatenate((absent_y, present_y), axis=None)

test_number_of_present = len([label for label in y_test if label == 1])
test_number_of_absent = len([label for label in y_test if label == 0])
print("Number of test genes:", len(y_test))
print("y stats: Number of present:", test_number_of_present," and number of absent:", test_number_of_absent)
print("Percentage of present:", test_number_of_present / len(y_test) * 100)

train_number_of_present = len([label for label in y_train_filter if label == 1])
train_number_of_absent = len([label for label in y_train_filter if label == 0])
print("Number of train genes:", len(y_train_filter))
print("y stats: Number of present:", train_number_of_present," and number of absent:", train_number_of_absent)

Number of test genes: 918
y stats: Number of present: 769  and number of absent: 149
Percentage of present: 83.76906318082789
Number of train genes: 4918
y stats: Number of present: 2459  and number of absent: 2459


In [62]:
# Fit Naive Bayes
model = MultinomialNB()
model.fit(X_train_filter, y_train_filter)

# Predict and evaluate
test_pred = model.predict(X_test)
print("FOR CountVectorizer")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision:", precision_score(y_test, test_pred))
print("Recall:", recall_score(y_test, test_pred))
print("F1-score:", f1_score(y_test, test_pred))
tf = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] == y_test[idx] and y_test[idx] == 1])
tn = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] == y_test[idx] and y_test[idx] == 0])
fp = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] != y_test[idx] and y_test[idx] == 0])
fn = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] != y_test[idx] and y_test[idx] == 1])
print("Number of true positives:", tf)
print("Number of true negatives:", tn)
print("Number of false positives:", fp)
print("Number of false negatives:", fn)
print("Negative precision (Out of all the negative predictions we made, how many were actually negative?):", tn/(tn+fn))
print("Negative recall (Out of all the sequences that should be predicted as unexpressed, how many did we correctly predict as unexpressed?):", tn/(tn+fp))

FOR CountVectorizer
Accuracy: 0.659041394335512
Precision: 0.9206642066420664
Recall: 0.6488946684005201
F1-score: 0.7612509534706331
Number of true positives: 499
Number of true negatives: 106
Number of false positives: 43
Number of false negatives: 270
Negative precision (Out of all the negative predictions we made, how many were actually negative?): 0.28191489361702127
Negative recall (Out of all the sequences that should be predicted as unexpressed, how many did we correctly predict as unexpressed?): 0.7114093959731543


In [63]:
# REPEATED FOR TFIDF
# Use CountVectorizer to count k-mers
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokenized_sequences)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

absent_idx = [idx for (idx, label) in enumerate(y_train) if label == 0]
present_idx = [idx for (idx, label) in enumerate(y_train) if label == 1][:len(absent_idx)]
absent_X = X_train[absent_idx]
present_X = X_train[present_idx]
absent_y = [l for l in y_train if l == 0]
present_y = [l for l in y_train if l == 1][:len(absent_y)]
X_train_filter = vstack([absent_X, present_X])
y_train_filter = np.concatenate((absent_y, present_y), axis=None)

test_number_of_present = len([label for label in y_test if label == 1])
test_number_of_absent = len([label for label in y_test if label == 0])
print("Number of test genes:", len(y_test))
print("y stats: Number of present:", test_number_of_present," and number of absent:", test_number_of_absent)
print("Percentage of present:", test_number_of_present / len(y_test) * 100)

train_number_of_present = len([label for label in y_train_filter if label == 1])
train_number_of_absent = len([label for label in y_train_filter if label == 0])
print("Number of train genes:", len(y_train_filter))
print("y stats: Number of present:", train_number_of_present," and number of absent:", train_number_of_absent)

Number of test genes: 918
y stats: Number of present: 769  and number of absent: 149
Percentage of present: 83.76906318082789
Number of train genes: 4918
y stats: Number of present: 2459  and number of absent: 2459


In [64]:
# Fit Naive Bayes
model = MultinomialNB()
model.fit(X_train_filter, y_train_filter)

# Predict and evaluate
test_pred = model.predict(X_test)
print("FOR TfidfVectorizer")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision:", precision_score(y_test, test_pred))
print("Recall:", recall_score(y_test, test_pred))
print("F1-score:", f1_score(y_test, test_pred))
tf = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] == y_test[idx] and y_test[idx] == 1])
tn = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] == y_test[idx] and y_test[idx] == 0])
fp = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] != y_test[idx] and y_test[idx] == 0])
fn = sum([1 for idx, label in enumerate(test_pred) if test_pred[idx] != y_test[idx] and y_test[idx] == 1])
print("Number of true positives:", tf)
print("Number of true negatives:", tn)
print("Number of false positives:", fp)
print("Number of false negatives:", fn)
print("Negative precision (Out of all the negative predictions we made, how many were actually negative?):", tn/(tn+fn))
print("Negative recall (Out of all the sequences that should be predicted as unexpressed, how many did we correctly predict as unexpressed?):", tn/(tn+fp))

FOR TfidfVectorizer
Accuracy: 0.6612200435729847
Precision: 0.922509225092251
Recall: 0.6501950585175552
F1-score: 0.7627765064836003
Number of true positives: 500
Number of true negatives: 107
Number of false positives: 42
Number of false negatives: 269
Negative precision (Out of all the negative predictions we made, how many were actually negative?): 0.2845744680851064
Negative recall (Out of all the sequences that should be predicted as unexpressed, how many did we correctly predict as unexpressed?): 0.7181208053691275
